## This notebook is to train an orientation model (regression or softmax)

### Note

Thi snotebook is written as if it is runnin gat the root directory of this project folder.  Therefore, the code below should be used to set the path to include the root project directory when this notebook is not in the root directory

```python
    import sys
    sys.path.insert(0, '../')
```
    

### Set Path

In [1]:
# set the relative path to the root directory
import sys

relative_root = '..\\'
sys.path.insert(0, relative_root)

### Load Tensorboard

In [2]:
# Load the TensorBoard notebook extension
# %load_ext tensorboard

### Imports

In [3]:
import os
import io
import time
import random
import itertools

import matplotlib.pyplot as plt
from copy import deepcopy
from tkinter import *
from tkinter import filedialog
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

import numpy as np
from pathlib import Path
from skimage.transform import resize
from collections import deque

from models.orientation import orientationModels as oM
from utils import io_utils as iou

# This is for the regresssion model
from utils.model_utils import rope_loss as loss_func
# from utils.model_utils import circle_loss as loss_func


### Set Path Parameters

In [4]:
loss_func_name = 'rope_loss'
# loss_func_name = 'circle_loss'

# Change dataset if required
dataset = 'dota'

usingRGB = True

# Set Model Type
# model_type = oM.SOFTMAX_TYPE
model_type = oM.REG_TYPE

# workstation specific
train_test_set_folder = Path('D:/workspaces/darcip_2020/data/orient/' + dataset + '/' + model_type)

In [5]:
# This notebook file may change and we use it for the summary string, so make sure the string is the same as the filename
# I don't feel like figuring out how to extract the name of this notebook as it is running!
notebook_name = 'TrainOrientationModel.ipynb'


channel_id = '_8band_'
img_channels = 8
if usingRGB:
    channel_id = '_rgb_'
    img_channels = 3

# setup file system to log results and save models
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = "cnn_test_results.txt"
log_dir_path = ''.join([relative_root, 'results\\CNNtest_', timestr])



# set paths to train and test files

x_train_file = train_test_set_folder / ('x_train' + channel_id + model_type + '_' + dataset + '.npy')
y_train_file = train_test_set_folder / ('y_train' + channel_id +  model_type + '_' + dataset +  '.npy')
x_test_file = train_test_set_folder / ('x_test' + channel_id + model_type + '_' + dataset +  '.npy')
y_test_file = train_test_set_folder / ('y_test' + channel_id + model_type + '_' + dataset +  '.npy')

# Setup Model Paramters
channels_first = False
data_format = 'channels_first'
if not channels_first:
    data_format = 'channels_last'


channels = img_channels
height, width = (70, 70)
patience = 50
num_epochs = 301
save_model_name = ''.join(['orientationModel_', 
                           dataset, '_',
                           str(channels), 'bands_', 
                           model_type, '_', 
                           str(height), 'x', str(width), '_',
                          str(num_epochs), '_epochs_', 
                          timestr, '.h5'])
save_model_folder = ''.join([relative_root, 'results/saved_models/'])

init_lr = 1e-2
batch_size = 64
img_rows = height
img_cols = width
momentum = 0.9
depth = channels

# fix random seed for reporducibility
np.random.seed(7)

if model_type == oM.SOFTMAX_TYPE:
    num_classes = 360
    class_names = [str(x) for x in range(360)] # only do this if we want to try the softmax degrees
elif model_type == oM.REG_TYPE:
    # This is nonsense for this model but we will enter values just to have them
    num_classes = 1
    class_names = ["Degrees_Normalized"]

### Create Run Summary String

In [6]:
summ_str = ''.join(['Date: ', timestr,
                    '\nTraining a ', model_type, ' Orientation Model:',
                    '\n\tFile:',
                    '\n\t\t', notebook_name,
                    '\n\tRunning Directory:',
                    '\n\t\t', os.getcwd(),
                    '\n\tLog Files In Folder: ',
                    '\n\t\t', log_dir_path,
                    '\n\tTraining and Test Files: ',
                    '\n\t\tx_train: ', str(x_train_file),
                    '\n\t\ty_train: ', str(y_train_file),
                    '\n\t\tx_test: ', str(x_test_file),
                    '\n\t\ty_test: ', str(y_test_file),
                    '\n\tSaved Model File: ',
                    '\n\t\t', save_model_folder, save_model_name,
                    '\n\tParameters:',
                    '\n\t\tLoss Function: ', loss_func_name , 
                    '\n\t\tImage (height, width): (', str(height), ', ', str(width), ')', 
                    '\n\t\tImage depth (i.e. bands): ', str(depth),
                    '\n\t\tpatience: ', str(patience),
                    '\n\t\tepochs: ', str(num_epochs),
                    '\n\t\tinit_lt: ', str(init_lr),
                    '\n\t\tbatch_size: ', str(batch_size),
                    '\n\t\timg_rows: ', str(img_rows),
                    '\n\t\timg_cols: ', str(img_cols),
                    '\n\t\tmomentum: ', str(momentum),
                    '\n'])

print(summ_str)

Date: 20200522-145745
Training a regression Orientation Model:
	File:
		TrainOrientationModel.ipynb
	Running Directory:
		D:\workspaces\darcip_2020\notebooks
	Log Files In Folder: 
		..\results\CNNtest_20200522-145745
	Training and Test Files: 
		x_train: D:\workspaces\darcip_2020\data\orient\dota\regression\x_train_rgb_regression_dota.npy
		y_train: D:\workspaces\darcip_2020\data\orient\dota\regression\y_train_rgb_regression_dota.npy
		x_test: D:\workspaces\darcip_2020\data\orient\dota\regression\x_test_rgb_regression_dota.npy
		y_test: D:\workspaces\darcip_2020\data\orient\dota\regression\y_test_rgb_regression_dota.npy
	Saved Model File: 
		..\results/saved_models/orientationModel_dota_3bands_regression_70x70_301_epochs_20200522-145745.h5
	Parameters:
		Loss Function: rope_loss
		Image (height, width): (70, 70)
		Image depth (i.e. bands): 3
		patience: 50
		epochs: 301
		init_lt: 0.01
		batch_size: 64
		img_rows: 70
		img_cols: 70
		momentum: 0.9



### Define Custom Callaback Functions

In [7]:
# tensorboard file writers
tb_logs = log_dir_path + '\\tensorboard_logs' 

if model_type == oM.SOFTMAX_TYPE:
    file_writer_cm = tf.summary.create_file_writer(tb_logs + '\\cm')

file_writer_imgs = tf.summary.create_file_writer(tb_logs + '\\images')
file_writer_thumb = tf.summary.create_file_writer(tb_logs + '\\thumbnails')

In [8]:
# Note: These function suse variables set outside of the function space

def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(x_test)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = confusion_matrix(y_test, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

    
def image_grid(image_list):
    """Return a 5x5 grid of the first 25 images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(10,10))
    for i in range(25):
        # Start next subplot.
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image_list[i], cmap=plt.cm.binary)

    return figure


def displayThumbnailGrid(image_list, epoch, max_outputs, title):
    # Prepare the plot
    images = []
    split_size = 25 # how many thumbs per image
    for i in range(0, len(image_list), split_size):
        figure = image_grid(image_list[i:i + split_size])
        img = plot_to_image(figure)
        images.append(deepcopy(img))
        
    # Convert to image and log
    with file_writer_thumb.as_default():
        tf.summary.image(title, images, max_outputs=int(max_outputs/split_size), step=epoch)
    
    
def display_sample_images(epoch, logs):
    # refresh the images after every X epochs
    X = 25
    max_outputs = 100
    if epoch % X == 0:
        with file_writer_imgs.as_default():
            training_samples = random.sample(list(x_train_load), max_outputs)
            displayThumbnailGrid(training_samples, epoch, max_outputs, "Sample Training Data Thumbnails")

            # Don't forget to reshape.
            x, y, c = x_train_load[0].shape
            images = np.reshape(training_samples, (-1, x, y, c))
            tf.summary.image("Training Data Examples", images, max_outputs=max_outputs, step=epoch)

        with file_writer_imgs.as_default():
            val_samples = random.sample(list(x_test_load), max_outputs)
            displayThumbnailGrid(val_samples, epoch, max_outputs, "Sample Validation Data Thumbnails")
            # Don't forget to reshape.
            x, y, c = x_test_load[0].shape
            images = np.reshape(val_samples, (-1, x, y, c))
            tf.summary.image("Validation Data Examples", images, max_outputs=max_outputs, step=epoch)
        
        
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
#     image = tf.expand_dims(image, 0)
    return image




### Create Directories

In [9]:
# create the required directory in the program root
# Note: We don't need thi sanymore since the tensorboard writer setups create the folder for us
# os.mkdir(log_dir_path)

### Setup Logging Support

In [10]:
log_file = os.path.join(log_dir_path, filename)
f = open(log_file, 'w')
original = sys.stdout
sys.stdout = iou.Tee(sys.stdout, f)

# print summary string so it gets into the log
print(summ_str)

Date: 20200522-145745
Training a regression Orientation Model:
	File:
		TrainOrientationModel.ipynb
	Running Directory:
		D:\workspaces\darcip_2020\notebooks
	Log Files In Folder: 
		..\results\CNNtest_20200522-145745
	Training and Test Files: 
		x_train: D:\workspaces\darcip_2020\data\orient\dota\regression\x_train_rgb_regression_dota.npy
		y_train: D:\workspaces\darcip_2020\data\orient\dota\regression\y_train_rgb_regression_dota.npy
		x_test: D:\workspaces\darcip_2020\data\orient\dota\regression\x_test_rgb_regression_dota.npy
		y_test: D:\workspaces\darcip_2020\data\orient\dota\regression\y_test_rgb_regression_dota.npy
	Saved Model File: 
		..\results/saved_models/orientationModel_dota_3bands_regression_70x70_301_epochs_20200522-145745.h5
	Parameters:
		Loss Function: rope_loss
		Image (height, width): (70, 70)
		Image depth (i.e. bands): 3
		patience: 50
		epochs: 301
		init_lt: 0.01
		batch_size: 64
		img_rows: 70
		img_cols: 70
		momentum: 0.9


Loaded The Training and Test Datase

7989/7989 [==============================] - ETA: 1:11 - loss: 0.2070 - rope_loss: 0.376 - ETA: 1:10 - loss: 0.2044 - rope_loss: 0.327 - ETA: 1:10 - loss: 0.2090 - rope_loss: 0.335 - ETA: 1:09 - loss: 0.2080 - rope_loss: 0.334 - ETA: 1:08 - loss: 0.2098 - rope_loss: 0.335 - ETA: 1:07 - loss: 0.2080 - rope_loss: 0.333 - ETA: 1:07 - loss: 0.2067 - rope_loss: 0.330 - ETA: 1:06 - loss: 0.2025 - rope_loss: 0.324 - ETA: 1:05 - loss: 0.2025 - rope_loss: 0.327 - ETA: 1:05 - loss: 0.2050 - rope_loss: 0.333 - ETA: 1:04 - loss: 0.2045 - rope_loss: 0.333 - ETA: 1:04 - loss: 0.2054 - rope_loss: 0.334 - ETA: 1:03 - loss: 0.2080 - rope_loss: 0.336 - ETA: 1:02 - loss: 0.2099 - rope_loss: 0.337 - ETA: 1:02 - loss: 0.2109 - rope_loss: 0.341 - ETA: 1:01 - loss: 0.2110 - rope_loss: 0.343 - ETA: 1:01 - loss: 0.2128 - rope_loss: 0.346 - ETA: 1:00 - loss: 0.2123 - rope_loss: 0.345 - ETA: 1:00 - loss: 0.2135 - rope_loss: 0.345 - ETA: 59s - loss: 0.2150 - rope_loss: 0.349 - ETA: 59s - loss: 0.2140 - rope_loss:

7989/7989 [==============================] - ETA: 1:13 - loss: 0.1418 - rope_loss: 0.214 - ETA: 1:12 - loss: 0.1552 - rope_loss: 0.235 - ETA: 1:10 - loss: 0.1642 - rope_loss: 0.250 - ETA: 1:10 - loss: 0.1620 - rope_loss: 0.248 - ETA: 1:09 - loss: 0.1649 - rope_loss: 0.244 - ETA: 1:08 - loss: 0.1604 - rope_loss: 0.243 - ETA: 1:08 - loss: 0.1583 - rope_loss: 0.241 - ETA: 1:07 - loss: 0.1652 - rope_loss: 0.257 - ETA: 1:06 - loss: 0.1674 - rope_loss: 0.264 - ETA: 1:06 - loss: 0.1650 - rope_loss: 0.260 - ETA: 1:05 - loss: 0.1655 - rope_loss: 0.262 - ETA: 1:05 - loss: 0.1630 - rope_loss: 0.258 - ETA: 1:04 - loss: 0.1639 - rope_loss: 0.260 - ETA: 1:03 - loss: 0.1656 - rope_loss: 0.264 - ETA: 1:03 - loss: 0.1643 - rope_loss: 0.260 - ETA: 1:02 - loss: 0.1634 - rope_loss: 0.257 - ETA: 1:01 - loss: 0.1642 - rope_loss: 0.260 - ETA: 1:01 - loss: 0.1642 - rope_loss: 0.259 - ETA: 1:00 - loss: 0.1649 - rope_loss: 0.260 - ETA: 1:00 - loss: 0.1642 - rope_loss: 0.258 - ETA: 59s - loss: 0.1642 - rope_loss

7989/7989 [==============================] - ETA: 1:11 - loss: 0.1875 - rope_loss: 0.302 - ETA: 1:10 - loss: 0.1822 - rope_loss: 0.290 - ETA: 1:10 - loss: 0.1808 - rope_loss: 0.288 - ETA: 1:09 - loss: 0.1737 - rope_loss: 0.279 - ETA: 1:09 - loss: 0.1737 - rope_loss: 0.282 - ETA: 1:09 - loss: 0.1730 - rope_loss: 0.277 - ETA: 1:08 - loss: 0.1713 - rope_loss: 0.273 - ETA: 1:07 - loss: 0.1681 - rope_loss: 0.266 - ETA: 1:07 - loss: 0.1692 - rope_loss: 0.267 - ETA: 1:06 - loss: 0.1660 - rope_loss: 0.263 - ETA: 1:06 - loss: 0.1659 - rope_loss: 0.263 - ETA: 1:05 - loss: 0.1628 - rope_loss: 0.254 - ETA: 1:04 - loss: 0.1607 - rope_loss: 0.252 - ETA: 1:04 - loss: 0.1614 - rope_loss: 0.254 - ETA: 1:03 - loss: 0.1602 - rope_loss: 0.252 - ETA: 1:02 - loss: 0.1623 - rope_loss: 0.257 - ETA: 1:02 - loss: 0.1603 - rope_loss: 0.252 - ETA: 1:01 - loss: 0.1591 - rope_loss: 0.251 - ETA: 1:01 - loss: 0.1584 - rope_loss: 0.250 - ETA: 1:00 - loss: 0.1595 - rope_loss: 0.250 - ETA: 59s - loss: 0.1587 - rope_loss

7989/7989 [==============================] - ETA: 1:11 - loss: 0.1613 - rope_loss: 0.290 - ETA: 1:09 - loss: 0.1641 - rope_loss: 0.282 - ETA: 1:09 - loss: 0.1551 - rope_loss: 0.246 - ETA: 1:09 - loss: 0.1474 - rope_loss: 0.229 - ETA: 1:08 - loss: 0.1475 - rope_loss: 0.226 - ETA: 1:08 - loss: 0.1453 - rope_loss: 0.218 - ETA: 1:08 - loss: 0.1455 - rope_loss: 0.221 - ETA: 1:07 - loss: 0.1437 - rope_loss: 0.218 - ETA: 1:06 - loss: 0.1425 - rope_loss: 0.216 - ETA: 1:06 - loss: 0.1411 - rope_loss: 0.215 - ETA: 1:05 - loss: 0.1396 - rope_loss: 0.213 - ETA: 1:04 - loss: 0.1378 - rope_loss: 0.211 - ETA: 1:04 - loss: 0.1380 - rope_loss: 0.212 - ETA: 1:03 - loss: 0.1391 - rope_loss: 0.214 - ETA: 1:02 - loss: 0.1389 - rope_loss: 0.212 - ETA: 1:02 - loss: 0.1404 - rope_loss: 0.216 - ETA: 1:01 - loss: 0.1397 - rope_loss: 0.215 - ETA: 1:01 - loss: 0.1392 - rope_loss: 0.214 - ETA: 1:00 - loss: 0.1392 - rope_loss: 0.214 - ETA: 59s - loss: 0.1388 - rope_loss: 0.213 - ETA: 59s - loss: 0.1402 - rope_loss:

...

7989/7989 [==============================] - ETA: 1:10 - loss: 0.0781 - rope_loss: 0.099 - ETA: 1:09 - loss: 0.0804 - rope_loss: 0.101 - ETA: 1:08 - loss: 0.0842 - rope_loss: 0.108 - ETA: 1:08 - loss: 0.0843 - rope_loss: 0.100 - ETA: 1:08 - loss: 0.0861 - rope_loss: 0.097 - ETA: 1:08 - loss: 0.0836 - rope_loss: 0.093 - ETA: 1:07 - loss: 0.0846 - rope_loss: 0.093 - ETA: 1:07 - loss: 0.0851 - rope_loss: 0.096 - ETA: 1:06 - loss: 0.0841 - rope_loss: 0.094 - ETA: 1:05 - loss: 0.0846 - rope_loss: 0.094 - ETA: 1:05 - loss: 0.0852 - rope_loss: 0.094 - ETA: 1:04 - loss: 0.0855 - rope_loss: 0.096 - ETA: 1:03 - loss: 0.0864 - rope_loss: 0.096 - ETA: 1:03 - loss: 0.0849 - rope_loss: 0.095 - ETA: 1:02 - loss: 0.0859 - rope_loss: 0.098 - ETA: 1:01 - loss: 0.0873 - rope_loss: 0.099 - ETA: 1:01 - loss: 0.0863 - rope_loss: 0.098 - ETA: 1:00 - loss: 0.0859 - rope_loss: 0.098 - ETA: 1:00 - loss: 0.0857 - rope_loss: 0.098 - ETA: 59s - loss: 0.0848 - rope_loss: 0.097 - ETA: 58s - loss: 0.0849 - rope_loss:

7989/7989 [==============================] - ETA: 1:10 - loss: 0.0851 - rope_loss: 0.110 - ETA: 1:09 - loss: 0.0780 - rope_loss: 0.093 - ETA: 1:09 - loss: 0.0788 - rope_loss: 0.092 - ETA: 1:10 - loss: 0.0865 - rope_loss: 0.103 - ETA: 1:10 - loss: 0.0916 - rope_loss: 0.104 - ETA: 1:10 - loss: 0.0923 - rope_loss: 0.110 - ETA: 1:09 - loss: 0.0937 - rope_loss: 0.116 - ETA: 1:08 - loss: 0.0930 - rope_loss: 0.114 - ETA: 1:07 - loss: 0.0941 - rope_loss: 0.115 - ETA: 1:06 - loss: 0.0946 - rope_loss: 0.117 - ETA: 1:05 - loss: 0.0921 - rope_loss: 0.113 - ETA: 1:05 - loss: 0.0926 - rope_loss: 0.115 - ETA: 1:04 - loss: 0.0933 - rope_loss: 0.116 - ETA: 1:03 - loss: 0.0926 - rope_loss: 0.115 - ETA: 1:03 - loss: 0.0925 - rope_loss: 0.115 - ETA: 1:02 - loss: 0.0918 - rope_loss: 0.114 - ETA: 1:02 - loss: 0.0909 - rope_loss: 0.113 - ETA: 1:01 - loss: 0.0906 - rope_loss: 0.111 - ETA: 1:01 - loss: 0.0899 - rope_loss: 0.110 - ETA: 1:00 - loss: 0.0904 - rope_loss: 0.111 - ETA: 59s - loss: 0.0895 - rope_loss

7989/7989 [==============================] - ETA: 1:12 - loss: 0.1119 - rope_loss: 0.104 - ETA: 1:12 - loss: 0.1029 - rope_loss: 0.095 - ETA: 1:10 - loss: 0.1019 - rope_loss: 0.107 - ETA: 1:08 - loss: 0.0922 - rope_loss: 0.096 - ETA: 1:07 - loss: 0.0906 - rope_loss: 0.098 - ETA: 1:07 - loss: 0.0924 - rope_loss: 0.099 - ETA: 1:06 - loss: 0.0921 - rope_loss: 0.101 - ETA: 1:06 - loss: 0.0898 - rope_loss: 0.099 - ETA: 1:05 - loss: 0.0904 - rope_loss: 0.101 - ETA: 1:05 - loss: 0.0880 - rope_loss: 0.098 - ETA: 1:05 - loss: 0.0880 - rope_loss: 0.100 - ETA: 1:04 - loss: 0.0888 - rope_loss: 0.100 - ETA: 1:03 - loss: 0.0880 - rope_loss: 0.099 - ETA: 1:03 - loss: 0.0874 - rope_loss: 0.099 - ETA: 1:02 - loss: 0.0883 - rope_loss: 0.100 - ETA: 1:02 - loss: 0.0882 - rope_loss: 0.101 - ETA: 1:01 - loss: 0.0878 - rope_loss: 0.101 - ETA: 1:01 - loss: 0.0888 - rope_loss: 0.104 - ETA: 1:00 - loss: 0.0887 - rope_loss: 0.103 - ETA: 59s - loss: 0.0902 - rope_loss: 0.104 - ETA: 59s - loss: 0.0896 - rope_loss:

7989/7989 [==============================] - ETA: 1:10 - loss: 0.0746 - rope_loss: 0.085 - ETA: 1:10 - loss: 0.0841 - rope_loss: 0.100 - ETA: 1:09 - loss: 0.0820 - rope_loss: 0.097 - ETA: 1:08 - loss: 0.0782 - rope_loss: 0.090 - ETA: 1:08 - loss: 0.0831 - rope_loss: 0.095 - ETA: 1:07 - loss: 0.0888 - rope_loss: 0.106 - ETA: 1:06 - loss: 0.0875 - rope_loss: 0.104 - ETA: 1:06 - loss: 0.0859 - rope_loss: 0.101 - ETA: 1:05 - loss: 0.0838 - rope_loss: 0.096 - ETA: 1:05 - loss: 0.0847 - rope_loss: 0.099 - ETA: 1:04 - loss: 0.0878 - rope_loss: 0.100 - ETA: 1:03 - loss: 0.0890 - rope_loss: 0.104 - ETA: 1:03 - loss: 0.0887 - rope_loss: 0.104 - ETA: 1:02 - loss: 0.0892 - rope_loss: 0.105 - ETA: 1:02 - loss: 0.0884 - rope_loss: 0.102 - ETA: 1:01 - loss: 0.0872 - rope_loss: 0.101 - ETA: 1:00 - loss: 0.0871 - rope_loss: 0.100 - ETA: 1:00 - loss: 0.0867 - rope_loss: 0.100 - ETA: 1:00 - loss: 0.0862 - rope_loss: 0.099 - ETA: 59s - loss: 0.0857 - rope_loss: 0.098 - ETA: 59s - loss: 0.0851 - rope_loss:

6784/7989 [========================>.....] - ETA: 1:11 - loss: 0.0712 - rope_loss: 0.076 - ETA: 1:11 - loss: 0.0690 - rope_loss: 0.072 - ETA: 1:10 - loss: 0.0768 - rope_loss: 0.091 - ETA: 1:09 - loss: 0.0820 - rope_loss: 0.095 - ETA: 1:08 - loss: 0.0781 - rope_loss: 0.089 - ETA: 1:08 - loss: 0.0781 - rope_loss: 0.090 - ETA: 1:07 - loss: 0.0775 - rope_loss: 0.089 - ETA: 1:06 - loss: 0.0767 - rope_loss: 0.087 - ETA: 1:06 - loss: 0.0781 - rope_loss: 0.087 - ETA: 1:05 - loss: 0.0810 - rope_loss: 0.089 - ETA: 1:04 - loss: 0.0814 - rope_loss: 0.089 - ETA: 1:04 - loss: 0.0822 - rope_loss: 0.091 - ETA: 1:03 - loss: 0.0827 - rope_loss: 0.088 - ETA: 1:03 - loss: 0.0838 - rope_loss: 0.090 - ETA: 1:02 - loss: 0.0839 - rope_loss: 0.092 - ETA: 1:02 - loss: 0.0860 - rope_loss: 0.096 - ETA: 1:01 - loss: 0.0873 - rope_loss: 0.099 - ETA: 1:00 - loss: 0.0878 - rope_loss: 0.101 - ETA: 1:00 - loss: 0.0863 - rope_loss: 0.098 - ETA: 59s - loss: 0.0880 - rope_loss: 0.099 - ETA: 59s - loss: 0.0886 - rope_loss:

### Prepare The Model

In [11]:
# Get the training and test data
x_train_load = np.load(x_train_file, allow_pickle=True)
y_train = np.load(y_train_file, allow_pickle=True)
x_test_load = np.load(x_test_file, allow_pickle=True)
y_test = np.load(y_test_file, allow_pickle=True)

if model_type == oM.SOFTMAX_TYPE:
    y_train = np.argmax(y_train, axis = 1)
    y_test = np.argmax(y_test, axis = 1)

print('\nLoaded The Training and Test Datasets')
print('\tx_train_load.shape = {}'.format(x_train_load.shape))
print('\tx_test_load.shape = {}'.format(x_test_load.shape))
print('\n\ty_train.shape = {}'.format(y_train.shape))
print('\ty_test.shape = {}'.format(y_test.shape))

## Adjust Data For Desired Depth

In [12]:
assert x_train_load.shape[1:] == x_test_load.shape[1:], 'Training and Test Set do not have same dimensions'
num_channels = x_train_load.shape[3]  # assume channels last
if channels_first:
    num_channels = x_train_load.shape[3]

if depth == 3 and num_channels == 8:
    rgb_from_8band = [4, 2, 1]
    if channels_first:
        x_train = x_train_load[:, rgb_from_8band, :, :]
        x_test = x_test_load[:, rgb_from_8band, :, :]
    else:
        x_train = x_train_load[:, :, :, rgb_from_8band]
        x_test = x_test_load[:, :, :, rgb_from_8band]
else:
    if channels_first:
        x_train = x_train_load[:, :depth, :, :]
        x_test = x_test_load[:, :depth, :, :]
    else:
        x_train = x_train_load[:, :, :, :depth]
        x_test = x_test_load[:, :, :, :depth]

print('\nReshaping Train and Test Data for Depth: {}'.format(depth))
print('\tx_train.shape = {}'.format(x_train.shape))
print('\tx_test.shape = {}'.format(x_test.shape))

In [13]:
# create the model
model_obj = oM.OrientationCnnModel(type=model_type, num_classes=num_classes)
model = model_obj.build(height, width, depth, data_format=data_format)

In [14]:
# compile model 
if model_type == oM.SOFTMAX_TYPE:
    optimizer = 'adam'
    model_obj.compile(optimizer=optimizer)
    loss_func_name = 'accuracy'
elif model_type == oM.REG_TYPE:
    optimizer = SGD(lr=init_lr, momentum=momentum, decay=init_lr / num_epochs)
    model_obj.compile(optimizer=optimizer, metrics=[loss_func])

In [15]:
# setup callbacks
img_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=display_sample_images)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_logs, histogram_freq=1, profile_batch=0)



callbacks = [tensorboard_callback, 
             img_callback,
             CSVLogger(os.path.join(log_dir_path, 'training.log')),
             EarlyStopping(monitor=loss_func_name, patience=patience),
             ModelCheckpoint(filepath=save_model_folder + save_model_name, monitor=loss_func_name, save_best_only=True)]

# Define the per-epoch callback.
# if model_type == oM.SOFTMAX_TYPE:
#     cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
#     callbacks.append(cm_callback)

## Train Model

In [ ]:
# Train the model
startTrainTime = time.time()

history = model.fit(x_train, 
                   y_train,
                   validation_data=(x_test, y_test),
                    epochs=num_epochs,
                    batch_size=batch_size,
                    callbacks=callbacks)

endTrainTime = time.time()
trainTime = endTrainTime - startTrainTime
print()
print('Total Training Time (sec): {}'.format(trainTime))

## Evaluate Model

In [ ]:
# %tensorboard --logdir log_dir_path

In [ ]:
# Load the best model file with file dialog
model = None

del model # ensure the previous model is deleted


root = Tk()
root.withdraw()
root.modelFilePath = filedialog.askopenfilename()
print(root.modelFilePath)

model = load_model(root.modelFilePath, custom_objects={loss_func_name: loss_func})


In [ ]:
# Evaluate the model to get the prediction w/the validation data
y_pred = model.predict(x_test)
y_pred_confTest = y_pred
y_pred = np.argmax(y_pred, axis = 1)
# y_true = np.argmax(y_test, axis = 1)
y_true = y_test
confMaxTest = np.max(y_pred_confTest, axis=1)

print('Pred shape: {}\t True shape: {}'.format(y_pred.shape, y_true.shape))

if model_type == oM.REG_TYPE:
    kappa = metrics.roc_curve(y_true, y_pred)
    print('kappa', kappa)

# save variables for later investigation 
np.save(os.path.join(log_dir_path, 'y_pred_test'), y_pred)
np.save(os.path.join(log_dir_path, 'ConfidenceScoresTest'), confMaxTest)

# print confusion matrix to screen
print(classification_report(y_true, y_pred, target_names=class_names))
cnf_matrix = confusion_matrix(y_true, y_pred)
print(confusion_matrix(y_true, y_pred))

### Scratch Area

In [ ]:
exit() # This releases the GPU(s)

In [ ]:
import math
deg = tf.convert_to_tensor(math.pi)

tf.math.cos(deg)

In [ ]:
# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
func_trace_log_path = tb_logs + '\\function_trace'
function_trace_writer = tf.summary.create_file_writer(func_trace_log_path)
# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = loss_func(x, y)
with function_trace_writer.as_default():
  tf.summary.trace_export(
      name="{} Function Trace".format(loss_func_name),
      step=0,
      profiler_outdir=func_trace_log_path)

In [ ]:
x_test_prep = np.expand_dims(x_test[0], axis=0)

In [ ]:
y_pred_1 = model.predict(x_test_prep)

In [ ]:
y_pred = model.predict(x_test)
# y_test_argmax = np.argmax(y_test, axis = 1)
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
cm = confusion_matrix(y_test, y_test)

In [ ]:
figure = plot_confusion_matrix(cm, class_names=class_names)

In [ ]:
y_pred.shape